In [1]:
import warnings
warnings.filterwarnings('ignore',category=RuntimeWarning)
import xarray as xr
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import glob,os,sys
from tqdm import tqdm
import datetime
import proplot as plot
from netCDF4 import Dataset
from wrf import getvar
import json,pickle
from scipy.ndimage import uniform_filter1d
from scipy.ndimage import gaussian_filter

sys.path.insert(1, '/work/08350/tg876493/stampede2/python_codes/2020_TC_CRF/dev/freddy0218/tools/')
import derive_var
import read_and_proc
%matplotlib inline
plot.rc.update({'figure.facecolor':'w','axes.labelweight':'ultralight',
                'tick.labelweight':'ultralight','gridminor.linestyle':'--','title.weight':'normal','linewidth':0.5})

In [2]:
#Function####################################################
def nearest_index(array, value):
    idx = (np.abs(array-value)).argmin()
    return idx.values
def forward_diff(arrayin=None,delta=None,axis=None):
    result = []
    if axis==0:
        for i in range(0,arrayin.shape[axis]-1):
            temp = (arrayin[i+1,:]-arrayin[i,:])/(delta)
            result.append(temp)
        return np.asarray(result)
    
def do_PCA(var=None,timezoom=None,smooth='Yes',gaussian=0.9,fromcenter='Yes',inradius=None,outradius=None,donormal='Yes',do_PCA='Yes',do_center='Yes'):
    """
    Var: Input variable (must have 4 dimensions! Time-pres-theta-radius)
    """
    if smooth=='Yes':
        from scipy.ndimage import gaussian_filter
        normal_var = []
        if donormal=='Yes':
            for presindex in range(len(var[0,:,0,0])):
                normal_var.append(gaussian_filter(normalize_inner(var[:,presindex,:,:],outradius,'Yes'),sigma=gaussian))
        elif donormal=='No':
            if do_center=='Yes':
                for presindex in range(len(var[0,:,0,0])):
                    normal_var.append(gaussian_filter(normalize_inner(var[:,presindex,:,:],outradius,'No','Yes'),sigma=gaussian))
            elif do_center=='No':
                for presindex in range(len(var[0,:,0,0])):
                    normal_var.append(gaussian_filter(normalize_inner(var[:,presindex,:,:],outradius,'No','No'),sigma=gaussian))  
        normal_var = np.swapaxes(np.asarray(normal_var),0,1)
    else:
        normal_var = []
        for presindex in range(len(var[0,:,0,0])):
            normal_var.append(normalize_inner(var[:,presindex,:,:],outradius))
        normal_var = np.swapaxes(np.asarray(normal_var),0,1)
    if fromcenter=='Yes':
        normal_varf = np.asarray([normal_var[i,:,:,:outradius].flatten() \
                                  for i in range(len(normal_var[timezoom[0]:timezoom[1],0,0,0]))])
    elif fromcenter=='No':
        normal_varf = np.asarray([normal_var[i,:,:,inradius:outradius].flatten() \
                                  for i in range(len(normal_var[timezoom[0]:timezoom[1],0,0,0]))])        
    #print("--Finish preprocesing--")
    if do_PCA=='Yes':
        from sklearn.decomposition import PCA
        import time
        start_time = time.time()
        skpcaVAR = PCA()
        skpcaVAR.fit(normal_varf.copy())
        #print("--- %s seconds ---" % (time.time() - start_time))
        return skpcaVAR,normal_var,normal_varf
    else:
        return normal_var,normal_varf

def normalize_inner(var=None,outerradius=None,standard='Yes',docenter='No'):
    PWper_ctrl = []
    for indx in range(len(var[:,0,0])):
        if docenter=='Yes':
            temp = var[indx,:,:outerradius]-np.nanmean(var[indx,:,:outerradius],axis=(0,1))
        elif docenter=='No':
            temp = var[indx,:,:outerradius]
        if standard=='Yes':
            PWper_ctrl.append((temp-np.nanmean(temp))/np.nanstd(temp))
        elif standard=='No':
            PWper_ctrl.append((temp))
    del temp
    return np.asarray(PWper_ctrl)

### Temperature

In [3]:
import gc
origin_path = '/scratch/06040/tg853394/tc/output/redux/maria/'

#############################################################
ctrlvar_dict = read_and_proc.read_some_azimuth_fields(fileloc=[origin_path+'ctl/azim_U_0-144hr.nc',origin_path+'ctl/azim_V_0-144hr.nc',origin_path+'ctl/azim_W_0-144hr.nc',origin_path+'ctl/azim_QVAPOR_0-144hr.nc'],
                                        fieldname=['U','V','W','QVAPOR'])
ncrf60var_dict = read_and_proc.read_some_azimuth_fields(fileloc=[origin_path+'ncrf_60h/azim_U_60-144hr.nc',origin_path+'ncrf_60h/azim_V_60-144hr.nc',origin_path+'ncrf_60h/azim_W_60-144hr.nc',origin_path+'ncrf_60h/azim_QVAPOR_60-144hr.nc'],
                                          fieldname=['U','V','W','QVAPOR'])
ncrf36var_dict = read_and_proc.read_some_azimuth_fields(fileloc=[origin_path+'lwcrf/azim_U_36-144hr.nc',origin_path+'lwcrf/azim_V_36-144hr.nc',origin_path+'lwcrf/azim_W_36-144hr.nc',origin_path+'lwcrf/azim_QVAPOR_36-144hr.nc'],
                                          fieldname=['U','V','W','QVAPOR'])
r300,r500=(nearest_index(ctrlvar_dict['U']['U'].radius,300),nearest_index(ctrlvar_dict['U']['U'].radius,500))
NCRF36_u,NCRF36_v,NCRF36_w,NCRF36_qv = read_and_proc.add_ctrl_before_senstart(ctrlvar_dict['U']['U'],ncrf36var_dict['U']['U'],'NCRF36','Yes'),\
read_and_proc.add_ctrl_before_senstart(ctrlvar_dict['V']['V'],ncrf36var_dict['V']['V'],'NCRF36','Yes'),read_and_proc.add_ctrl_before_senstart(ctrlvar_dict['W']['W'],ncrf36var_dict['W']['W'],'NCRF36','Yes'),\
read_and_proc.add_ctrl_before_senstart(ctrlvar_dict['QVAPOR']['QVAPOR'],ncrf36var_dict['QVAPOR']['QVAPOR'],'NCRF36','Yes')
NCRF60_u,NCRF60_v,NCRF60_w,NCRF60_qv = read_and_proc.add_ctrl_before_senstart(ctrlvar_dict['U']['U'],ncrf60var_dict['U']['U'],'NCRF60','Yes'),\
read_and_proc.add_ctrl_before_senstart(ctrlvar_dict['V']['V'],ncrf60var_dict['V']['V'],'NCRF60','Yes'),read_and_proc.add_ctrl_before_senstart(ctrlvar_dict['W']['W'],ncrf60var_dict['W']['W'],'NCRF60','Yes'),\
read_and_proc.add_ctrl_before_senstart(ctrlvar_dict['QVAPOR']['QVAPOR'],ncrf60var_dict['QVAPOR']['QVAPOR'],'NCRF60','Yes')

In [4]:
import gc
##############################################################################################################
# Radial and Tangential Velocities
##############################################################################################################
ctrlhUrad,ctrlhVtan,_ = derive_var.ruppert_vtmax_calc(ctrlvar_dict['U']['U'].data,ctrlvar_dict['V']['V'].data,r500,ctrlvar_dict['U']['U'][:,0,:,:].azmiuth.values,'NCRF36')
ncrf_36hUrad,ncrf_36hVtan,_ = derive_var.ruppert_vtmax_calc(NCRF36_u,NCRF36_v,r500,ctrlvar_dict['U']['U'][:,0,:,:].azmiuth.values,'NCRF36')
ncrf_60hUrad,ncrf_60hVtan,_ = derive_var.ruppert_vtmax_calc(NCRF60_u[:],NCRF60_v[:],r500,ctrlvar_dict['U']['U'][:,0,:,:].azmiuth.values,'NCRF60')

del NCRF36_u,NCRF60_u,NCRF36_v,NCRF60_v,ctrlvar_dict['U'],ctrlvar_dict['V'],ncrf36var_dict['U'],ncrf36var_dict['V'],ncrf60var_dict['U'],ncrf60var_dict['V']
gc.collect()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:39<00:00,  1.02s/it]


83

##### Ugrad,Vgrad,Wgrad

In [5]:
ctrl_ugrad_b = forward_diff(gaussian_filter(np.swapaxes(np.asarray(ctrlhUrad),0,1),sigma=[3,0,0,0]),60*60,0)
ctrl_vgrad_b = forward_diff(gaussian_filter(np.swapaxes(np.asarray(ctrlhVtan),0,1),sigma=[3,0,0,0]),60*60,0)
ctrl_wgrad_b = forward_diff(gaussian_filter(ctrlvar_dict['W']['W'].data,sigma=[3,0,0,0]),60*60,0)
del ctrlhUrad,ctrlhVtan,ncrf_36hUrad,ncrf_36hVtan,ncrf_60hUrad,ncrf_60hVtan,ctrlvar_dict['W']['W'],ncrf60var_dict,ncrf36var_dict
gc.collect()

40

### Radiative

In [3]:
import gc
origin_path = '/scratch/06040/tg853394/tc/output/redux/maria/'

#############################################################
ctrlvar_dict = read_and_proc.read_some_azimuth_fields(fileloc=[origin_path+'ctl/azim_RTHRATLW_0-144hr.nc',origin_path+'ctl/azim_RTHRATSW_0-144hr.nc',origin_path+'ctl/azim_RTHRATLWC_0-144hr.nc',origin_path+'ctl/azim_RTHRATSWC_0-144hr.nc'],
                                        fieldname=['RTHRATLW','RTHRATSW','RTHRATLWC','RTHRATSWC'])
ncrf60var_dict = read_and_proc.read_some_azimuth_fields(fileloc=[origin_path+'ncrf_60h/azim_RTHRATLW_60-144hr.nc',origin_path+'ncrf_60h/azim_RTHRATSW_60-144hr.nc',origin_path+'ncrf_60h/azim_RTHRATLWC_60-144hr.nc',origin_path+'ncrf_60h/azim_RTHRATSWC_60-144hr.nc'],
                                          fieldname=['RTHRATLW','RTHRATSW','RTHRATLWC','RTHRATSWC'])
lwcrfvar_dict = read_and_proc.read_some_azimuth_fields(fileloc=[origin_path+'lwcrf/azim_RTHRATLW_36-144hr.nc',origin_path+'lwcrf/azim_RTHRATSW_36-144hr.nc',origin_path+'lwcrf/azim_RTHRATLWC_36-144hr.nc',origin_path+'lwcrf/azim_RTHRATSWC_36-144hr.nc'],
                                          fieldname=['RTHRATLW','RTHRATSW','RTHRATLWC','RTHRATSWC'])
ncrf36var_dict = read_and_proc.read_some_azimuth_fields(fileloc=[origin_path+'ncrf_36h/azim_RTHRATLW_36-144hr.nc',origin_path+'ncrf_36h/azim_RTHRATSW_36-144hr.nc',origin_path+'ncrf_36h/azim_RTHRATLWC_36-144hr.nc',origin_path+'ncrf_36h/azim_RTHRATSWC_36-144hr.nc'],
                                          fieldname=['RTHRATLW','RTHRATSW','RTHRATLWC','RTHRATSWC'])
r300,r500=(nearest_index(ctrlvar_dict['RTHRATLW']['RTHRATLW'].radius,300),nearest_index(ctrlvar_dict['RTHRATLW']['RTHRATLW'].radius,500))
NCRF36_LW,NCRF36_SW,NCRF36_LWC,NCRF36_SWC = read_and_proc.add_ctrl_before_senstart(ctrlvar_dict['RTHRATLW']['RTHRATLW'],ncrf36var_dict['RTHRATLW']['RTHRATLW'],'NCRF36','Yes'),\
read_and_proc.add_ctrl_before_senstart(ctrlvar_dict['RTHRATSW']['RTHRATSW'],ncrf36var_dict['RTHRATSW']['RTHRATSW'],'NCRF36','Yes'),read_and_proc.add_ctrl_before_senstart(ctrlvar_dict['RTHRATLWC']['RTHRATLWC'],ncrf36var_dict['RTHRATLWC']['RTHRATLWC'],'NCRF36','Yes'),\
read_and_proc.add_ctrl_before_senstart(ctrlvar_dict['RTHRATSWC']['RTHRATSWC'],ncrf36var_dict['RTHRATSWC']['RTHRATSWC'],'NCRF36','Yes')
LWCRF_LW,LWCRF_SW,LWCRF_LWC,LWCRF_SWC = read_and_proc.add_ctrl_before_senstart(ctrlvar_dict['RTHRATLW']['RTHRATLW'],lwcrfvar_dict['RTHRATLW']['RTHRATLW'],'NCRF36','Yes'),\
read_and_proc.add_ctrl_before_senstart(ctrlvar_dict['RTHRATSW']['RTHRATSW'],lwcrfvar_dict['RTHRATSW']['RTHRATSW'],'NCRF36','Yes'),read_and_proc.add_ctrl_before_senstart(ctrlvar_dict['RTHRATLWC']['RTHRATLWC'],lwcrfvar_dict['RTHRATLWC']['RTHRATLWC'],'NCRF36','Yes'),\
read_and_proc.add_ctrl_before_senstart(ctrlvar_dict['RTHRATSWC']['RTHRATSWC'],lwcrfvar_dict['RTHRATSWC']['RTHRATSWC'],'NCRF36','Yes')
NCRF60_LW,NCRF60_SW,NCRF60_LWC,NCRF60_SWC = read_and_proc.add_ctrl_before_senstart(ctrlvar_dict['RTHRATLW']['RTHRATLW'],ncrf60var_dict['RTHRATLW']['RTHRATLW'],'NCRF60','Yes'),\
read_and_proc.add_ctrl_before_senstart(ctrlvar_dict['RTHRATSW']['RTHRATSW'],ncrf60var_dict['RTHRATSW']['RTHRATSW'],'NCRF60','Yes'),read_and_proc.add_ctrl_before_senstart(ctrlvar_dict['RTHRATLWC']['RTHRATLWC'],ncrf60var_dict['RTHRATLWC']['RTHRATLWC'],'NCRF60','Yes'),\
read_and_proc.add_ctrl_before_senstart(ctrlvar_dict['RTHRATSWC']['RTHRATSWC'],ncrf60var_dict['RTHRATSWC']['RTHRATSWC'],'NCRF60','Yes')

In [4]:
radtype = 'ir'
if radtype=='all':
    ctrl_rad = ctrlvar_dict['RTHRATLW']['RTHRATLW']+ctrlvar_dict['RTHRATSW']['RTHRATSW']
    ncrf36_rad = NCRF36_LW+NCRF36_SW
    lwcrf_rad = LWCRF_LW+LWCRF_SW
    ncrf60_rad = NCRF60_LW+NCRF60_SW
elif radtype=='ir':
    ctrl_rad = ctrlvar_dict['RTHRATLW']['RTHRATLW']-ctrlvar_dict['RTHRATLWC']['RTHRATLWC']
    ncrf36_rad = NCRF36_LW-NCRF36_LWC
    lwcrf_rad = LWCRF_LW-LWCRF_LWC
    ncrf60_rad = NCRF60_LW-NCRF60_LWC    
del ncrf60var_dict,ncrf36var_dict,lwcrfvar_dict,LWCRF_LW,LWCRF_SW,LWCRF_LWC,LWCRF_SWC,NCRF36_LW,NCRF36_SW,NCRF36_LWC,NCRF36_SWC,NCRF60_LW,NCRF60_SW,NCRF60_LWC,NCRF60_SWC
gc.collect()

177

### Remove diurnal effect

In [ ]:
from scipy.ndimage import uniform_filter1d
##############################################################################################
# (a) Urad, Vtan
UVdict = {}
dictname = ['ctrlU','ctrlV','ncrf36U','ncrf36V','ncrf60U','ncrf60V']
for indx,obj in tqdm(enumerate([ctrlhUrad,ctrlhVtan,ncrf_36hUrad,ncrf_36hVtan,ncrf_60hUrad,ncrf_60hVtan])):
    UVdict[dictname[indx]] = uniform_filter1d(np.swapaxes(np.asarray(obj),0,1),size=24,axis=0)
del ctrlhUrad,ctrlhVtan,ncrf_36hUrad,ncrf_36hVtan,ncrf_60hUrad,ncrf_60hVtan,dictname
gc.collect()
##############################################################################################
# (b) W, QV
WQVdict = {}
dictname = ['ctrlW','ctrlQV','ncrf36W','ncrf36QV','ncrf60W','ncrf60QV']
for indx,obj in tqdm(enumerate([ctrlvar_dict['W']['W'],ctrlvar_dict['QVAPOR']['QVAPOR'],NCRF36_w,NCRF36_qv,NCRF60_w,NCRF60_qv])):
    WQVdict[dictname[indx]] = uniform_filter1d((obj),size=24,axis=0)
del NCRF36_w,NCRF36_qv,NCRF60_w,NCRF60_qv,ctrlvar_dict,ncrf36var_dict,ncrf60var_dict
gc.collect()

##### Potential Temperature

In [5]:
path = '/work2/08350/tg876493/stampede2/python_codes/tcg_radiation/output/'
file = [path+'ctrl_thetaall',path+'ncrf36_thetaall',path+'ncrf60_thetaall',path+'lwcrf_thetaall']

output = []
for i in range(len(file)):
    with open(file[i],'rb') as f:
        output.append(pickle.load(f))

theta_dict = {'CTRL_theta':output[0],'NCRF36_theta':output[1],'NCRF60_theta':output[2],'LWCRF_theta':output[3]}
# To be consistent with azimuthal files
########################################################################################################
ctrl_thetaA,ncrf36_thetaA,ncrf60_thetaA,lwcrf_thetaA = np.swapaxes(np.asarray(theta_dict['CTRL_theta']),0,1),\
np.swapaxes(np.asarray(theta_dict['NCRF36_theta']),0,1),np.swapaxes(np.asarray(theta_dict['NCRF60_theta']),0,1),\
np.swapaxes(np.asarray(theta_dict['LWCRF_theta']),0,1)

ncrf36_thetaA_todo = read_and_proc.add_ctrl_before_senstart(ctrl_thetaA,ncrf36_thetaA,'NCRF36','Yes')
lwcrf_thetaA_todo = read_and_proc.add_ctrl_before_senstart(ctrl_thetaA,lwcrf_thetaA,'NCRF36','Yes')
ncrf60_thetaA_todo = read_and_proc.add_ctrl_before_senstart(ctrl_thetaA,ncrf60_thetaA,'NCRF60','Yes')
del ncrf36_thetaA,ncrf60_thetaA,lwcrf_thetaA
gc.collect()
###################################################################
# Heating Rate
###################################################################
buoy='No'
rad = 'Yes'
if buoy=='Yes':
    ctrl_thetagrad_b = forward_diff(gaussian_filter(ctrl_buoyA,sigma=[3,0,0,0]),60*60,0)
else:
    if rad=='Yes':
        ctrl_thetagrad_b = forward_diff(gaussian_filter(ctrl_thetaA,sigma=[3,0,0,0]),60*60,0)-gaussian_filter(ctrl_rad,sigma=[3,0,0,0])[1:,:]
        ncrf36_thetagrad_b = forward_diff(gaussian_filter(ncrf36_thetaA_todo,sigma=[3,0,0,0]),60*60,0)-gaussian_filter(ncrf36_rad,sigma=[3,0,0,0])[1:,:]
        lwcrf_thetagrad_b = forward_diff(gaussian_filter(lwcrf_thetaA_todo,sigma=[3,0,0,0]),60*60,0)-gaussian_filter(lwcrf_rad,sigma=[3,0,0,0])[1:,:]
        ncrf60_thetagrad_b = forward_diff(gaussian_filter(ncrf60_thetaA_todo,sigma=[3,0,0,0]),60*60,0)-gaussian_filter(ncrf60_rad,sigma=[3,0,0,0])[1:,:]
    else:
        ctrl_thetagrad_b = forward_diff(gaussian_filter(ctrl_thetaA,sigma=[3,0,0,0]),60*60,0)
        ncrf36_thetagrad_b = forward_diff(gaussian_filter(ncrf36_thetaA_todo,sigma=[3,0,0,0]),60*60,0)
        ncrf60_thetagrad_b = forward_diff(gaussian_filter(ncrf60_thetaA_todo,sigma=[3,0,0,0]),60*60,0)        
del ctrl_thetaA,ncrf36_thetaA_todo,ncrf60_thetaA_todo,theta_dict
gc.collect()

#ctrl_noIR = ctrl_thetagrad_b-gaussian_filter(ctrl_IR,sigma=[3,0,0,0])[1:]
#ncrf36_noIR = ncrf36_thetagrad_b-gaussian_filter(ncrf36_IR,sigma=[3,0,0,0])[1:]
#ncrf60_noIR = ncrf60_thetagrad_b-gaussian_filter(ncrf60_IR,sigma=[3,0,0,0])[1:]

0

In [6]:
from scipy.ndimage import uniform_filter1d
##############################################################################################
# (c) dthdt
DTHDTdict = {}
dictname = ['ctrldthdt','ncrf36dthdt','ncrf60dthdt','lwcrfdthdt']
for indx,obj in tqdm(enumerate([ctrl_thetagrad_b,ncrf36_thetagrad_b,ncrf60_thetagrad_b,lwcrf_thetagrad_b])):
    DTHDTdict[dictname[indx]] = uniform_filter1d((obj),size=24,axis=0)
del ctrl_thetagrad_b,ncrf36_thetagrad_b,ncrf60_thetagrad_b,lwcrf_thetagrad_b
gc.collect()

#(d) Gradient kinematics
##############################################################################################
#WINDGdict = {}
#dictname = ['ctrlUG','ctrlVG','ctrlWG']
#for indx,obj in tqdm(enumerate([ctrl_ugrad_b,ctrl_vgrad_b,ctrl_wgrad_b])):
#    WINDGdict[dictname[indx]] = uniform_filter1d((obj),size=24,axis=0)
#del ctrl_ugrad_b,ctrl_vgrad_b,ctrl_wgrad_b
#gc.collect()

4it [03:47, 56.78s/it]


18

#### PCA

In [7]:
###################################################################
# Cut off time
###################################################################
ctrlvar_dict = read_and_proc.read_some_azimuth_fields(fileloc=[origin_path+'ctl/azim_U_0-144hr.nc'],fieldname=['U'])
ctrl_t2,ctrl_t4 = nearest_index(ctrlvar_dict['U']['U'].time/24,1)-1,nearest_index(ctrlvar_dict['U']['U'].time/24,5)-1
r300,r500=nearest_index(ctrlvar_dict['U']['U'].radius,300),nearest_index(ctrlvar_dict['U']['U'].radius,500)
###################################################################
# Time difference: gaussian=[0,0,0]; Other: gaussian=[3,0,0]
###################################################################
#CTRL_PCAdict,CTRL_flatvardict = {},{}
#varname = ['U','V','W','QV','dthdt']
#for indx,obj in tqdm(enumerate([UVdict['ctrlU'],UVdict['ctrlV'],WQVdict['ctrlW'],WQVdict['ctrlQV'],DTHDTdict['ctrldthdt']])):
#    temp1,_,temp2 = do_PCA(var=obj,timezoom=[ctrl_t2,ctrl_t4],smooth='Yes',
#                           gaussian=[0,0,0],fromcenter='Yes',inradius=None,outradius=r500,donormal='No',do_PCA='Yes',do_center='No')
#    CTRL_PCAdict[varname[indx]] = temp1
#    CTRL_flatvardict[varname[indx]] = temp2
#    del temp1,temp2
#    gc.collect()
#del UVdict['ctrlU'],UVdict['ctrlV'],WQVdict['ctrlW'],WQVdict['ctrlQV'],DTHDTdict['ctrldthdt']
#gc.collect()
####################################################################
# Output radiative terms/WG terms
CTRL_flatvardict = {}
varname = ['ctrlNORAD','ncrf36NORAD','ncrf60NORAD','lwcrfNORAD']
for indx,obj in tqdm(enumerate([DTHDTdict['ctrldthdt'],DTHDTdict['ncrf36dthdt'],DTHDTdict['ncrf36dthdt'],DTHDTdict['lwcrfdthdt']])):
    _,temp2 = do_PCA(var=obj,timezoom=[ctrl_t2,ctrl_t4],smooth='Yes',
                           gaussian=[0,0,0],fromcenter='Yes',inradius=None,outradius=r500,donormal='No',do_PCA='No',do_center='No')
    CTRL_flatvardict[varname[indx]] = temp2
    del temp2
    gc.collect()
del DTHDTdict
gc.collect()
####################################################################
# Output full terms for sensitivity experiments
####################################################################
#NCRF36_flatvardict,NCRF60_flatvardict = {},{}
#varname = ['U','V','W','QV','dthdt']
#for indx,obj in tqdm(enumerate([UVdict['ncrf36U'],UVdict['ncrf36V'],WQVdict['ncrf36W'],WQVdict['ncrf36QV'],DTHDTdict['ncrf36dthdt']])):
#    _,temp2 = do_PCA(var=obj,timezoom=[ctrl_t2,ctrl_t4],smooth='Yes',
#                           gaussian=[0,0,0],fromcenter='Yes',inradius=None,outradius=r500,donormal='No',do_PCA='No',do_center='No')
#    NCRF36_flatvardict[varname[indx]] = temp2
#    del temp2
#    gc.collect()
#del UVdict['ncrf36U'],UVdict['ncrf36V'],WQVdict['ncrf36W'],WQVdict['ncrf36QV'],DTHDTdict['ncrf36dthdt']
#gc.collect()

#for indx,obj in tqdm(enumerate([UVdict['ncrf60U'],UVdict['ncrf60V'],WQVdict['ncrf60W'],WQVdict['ncrf60QV'],DTHDTdict['ncrf60dthdt']])):
#    _,temp2 = do_PCA(var=obj,timezoom=[ctrl_t2,ctrl_t4],smooth='Yes',
#                           gaussian=[0,0,0],fromcenter='Yes',inradius=None,outradius=r500,donormal='No',do_PCA='No',do_center='No')
#    NCRF60_flatvardict[varname[indx]] = temp2
#    del temp2
#    gc.collect()
#del UVdict['ncrf60U'],UVdict['ncrf60V'],WQVdict['ncrf60W'],WQVdict['ncrf60QV'],DTHDTdict['ncrf60dthdt']
#gc.collect()

4it [00:14,  3.70s/it]


0

In [8]:
import json,pickle
def save_to_pickle(loc=None,var=None):
    with open(loc,"wb") as f:
        pickle.dump(var,f)
    return None

#!mkdir outputCTRL_PCAdict,CTRL_flatvardict
#save_to_pickle(loc='../output/1010_nc_nodiurnal/CTRL_PCAdict',var=CTRL_PCAdict)
save_to_pickle(loc='../output/1010_nc_nodiurnal/NOIR_flatvardict',var=CTRL_flatvardict)
#save_to_pickle(loc='../output/1010_nc_nodiurnal/LWCRF_flatvardict',var=NCRF36_flatvardict)
#save_to_pickle(loc='../output/1010_nc_nodiurnal/NCRF60_flatvardict',var=NCRF60_flatvardict)